# U-NET FOR PARAMOS IN THE WORLD

In this notebook we will use the U-Net architecture to segment the paramos in the world. 

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext autoreload

### Cuda info 

In [ ]:
!nvidia-smi

### Libraries

In [ ]:
import sagemaker
import os
import boto3
import sys
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
print("Region = {}".format(region))

## Import S3 bucket and Data Files

In [ ]:
s3 = boto3.resource('s3')

bucket_name = 'xxxxxxxxxxxxx'

def download_s3_folders(bucket_name, prefixes):
    bucket = s3.Bucket(bucket_name)
    for prefix in prefixes:
        for obj in bucket.objects.filter(Prefix=prefix):
            destination_path = os.path.join('..', obj.key)
            if not os.path.exists(os.path.dirname(destination_path)):
                os.makedirs(os.path.dirname(destination_path))
            bucket.download_file(obj.key, destination_path)

download_s3_folders(bucket_name, ['config/', 'data/', 'packages/', 'models/'])

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import packages.utils.get_path as path
import config.constants as const
ABSOLUTE_PATH = path.make_path()

In [ ]:
data_val_path = path.make_path(ABSOLUTE_PATH, const.DATA_DIR, const.PROCESSED_DIR, const.VALIDATION_DIR)
normalized_path = path.make_path(ABSOLUTE_PATH, const.DATA_DIR, const.PROCESSED_DIR, const.NORMALIZATION_DIR)

In [ ]:
import numpy as np

eval_rgb_images = np.load("{}/{}".format(data_val_path, const.VAL_RGB_NPY))
eval_label_images = np.load("{}/{}".format(data_val_path, const.VAL_LABEL_NPY))


In [ ]:
eval_label_images = eval_label_images / 255.0
eval_label_images = np.mean(eval_label_images, axis=-1, keepdims=True)
eval_label_images = (eval_label_images > 0.5).astype(np.float32)

In [ ]:
#Z-score normalization
mean = np.load("{}/{}".format(normalized_path, const.MEAN_NPY))
std = np.load("{}/{}".format(normalized_path, const.STD_NPY))

eval_rgb_images = (eval_rgb_images - mean) / std

In [ ]:
import tarfile
tar = tarfile.open(ABSOLUTE_PATH + "/models/model.tar.gz", "r:gz")
tar.extractall(path=ABSOLUTE_PATH + "/models/")
tar.close()

In [ ]:
from tensorflow.keras.models import load_model

model = load_model(ABSOLUTE_PATH + f"/models")

# UNET Model Inference

In [ ]:
predictions = model.predict(eval_rgb_images)

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Flatten the arrays for sklearn
true_labels = eval_label_images.flatten()
predicted_labels = np.round(predictions).flatten()  # round predictions to 0 or 1

print(classification_report(true_labels, predicted_labels))

In [ ]:
predictions = prediction.predict(eval_rgb_images)

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import random
# random numer in eval_label_images  length
index = random.randint(0, len(eval_label_images) - 1) #43 ,482, 596
print(index)
test_image = eval_rgb_images[index]
true_label = eval_label_images[index]
predicted_label = predictions[index]

# Plot the test image and the masks
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(test_image)
axes[0].set_title('Test Image')
axes[1].imshow(true_label.squeeze(), cmap='gray')
axes[1].set_title('True Label')
axes[2].imshow(predicted_label.squeeze(), cmap='gray')
axes[2].set_title('Predicted Label')

html = """
<div style='text-align: center;'>
    <div style='display: inline-block; padding: 10px; background-color: black; color: white; border-radius: 5px; margin: 5px; font-size: 15px; font-weight: bold; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5); font-family: Arial, sans-serif;'>Black is 'paramo'</div>
    <div style='display: inline-block; padding: 10px; background-color: white; color: black; border: 1px solid black; border-radius: 5px; margin: 5px; font-size: 15px; font-weight: bold; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5); font-family: Arial, sans-serif;'>White is 'not paramo'</div>
</div>
"""

# Display the HTML snippet
display(HTML(html))
plt.show()

In [ ]:
from sagemaker.serverless import ServerlessInferenceConfig

In [ ]:
serverless_config = ServerlessInferenceConfig(
  memory_size_in_mb=1024,
  max_concurrency=5,
)

In [ ]:
from sagemaker.serializers import NumpySerializer
unet_deploy = model.deploy(endpoint_name='unet_endpoint_serverless',
                           serverless_inference_config=serverless_config,
                           serializer=NumpySerializer())